In [1]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import SimpleITK as sitk
from types import SimpleNamespace
import yaml
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
import numpy.random as random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.utils import flow_to_image
from PIL import Image

In [3]:
class RayDataset(Dataset):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.batch_size = args.batch_size

        fixed = sitk.ReadImage(args.fixed_vol_path) # fixed_vol_path is the path to the fixed volume
        self.props = [
            fixed.GetSize(),
            fixed.GetOrigin(),
            fixed.GetSpacing(),
            fixed.GetDirection(),
        ]
        fixed = sitk.GetArrayFromImage(fixed) # fixed is the fixed volume
        
        self.fixed = torch.from_numpy(
            (fixed - args.min_val) / args.std_val
        ) # fixed is the fixed volume after normalization

        self.H, self.D, self.W = self.fixed.shape # H is the height, D is the depth, W is the width
        self.vol_size = self.H * self.D * self.W # vol_size is the total number of voxels in the fixed volume
        self.vol_shape = torch.tensor(([self.W, self.D, self.H])) # vol_shape is the shape of the fixed volume

        index_h = torch.linspace(-1.0, 1.0, self.H + 1)[:-1] + 1 / self.H # index_h is the height index
        index_d = torch.linspace(-1.0, 1.0, self.D + 1)[:-1] + 1 / self.D # index_d is the depth index
        index_w = torch.linspace(-1.0, 1.0, self.W + 1)[:-1] + 1 / self.W # index_w is the width index
        grid_h, grid_d, grid_w = torch.meshgrid(
            [index_h, index_d, index_w], indexing="ij"
        ) # grid_h, grid_d, grid_w are the height, depth, and width grids
        pts = torch.stack([grid_w, grid_d, grid_h], -1) # pts is the grid of the fixed volume
        self.pts = pts

        self.indexs = np.repeat(np.arange(self.vol_size), args.num_repeat)

        self.pts_vec = self.pts.view(-1, 3)
        self.vals_vec = self.fixed.view(-1, 1)

    def __len__(self):
        return self.vol_size // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size

        indexs = self.indexs[start: end]
        pts = self.pts_vec[indexs]
        vals = self.vals_vec[indexs]

        return dict(pts=pts, vals=vals)

    def shuffle(self):
        random.shuffle(self.indexs)

    def random_plane(self, view="axial"):
        if view == "axial":
            index = random.choice(range(self.H))
            pts = self.pts[index, :, :]
            vals = self.fixed[index, :, :]
        elif view == "sagital":
            index = random.choice(range(self.D))
            # index = 278
            pts = self.pts[:, index, :]
            vals = self.fixed[:, index, :]
        elif view == "coronal":
            index = random.choice(range(self.W))
            pts = self.pts[:, :, index]
            vals = self.fixed[:, :, index]
        return pts, vals, index

    def all_planes(self):
        return self.pts, self.fixed

    def tensor2vol(self, tensor, dvf=False):
        if dvf:
            tensor = tensor * self.vol_shape.to(tensor.device)
        else:
            tensor = tensor * self.args.std_val + self.args.min_val
        array = tensor.cpu().numpy()
        vol = sitk.GetImageFromArray(array)
        if dvf:
            vol = sitk.Cast(vol, sitk.sitkVectorFloat32)
        else:
            vol = sitk.Cast(vol, sitk.sitkInt16)
        vol.SetOrigin(self.props[1])
        vol.SetSpacing(self.props[2])
        vol.SetDirection(self.props[3])
        return vol


In [4]:
class Embedder(nn.Module):
    def __init__(self, input_ch, inc_input, max_freq, N_freqs):
        super().__init__()
        self.freq_bands = 2.0 ** torch.linspace(0.0, max_freq, steps=N_freqs)
        self.out_dim = 2 * N_freqs * input_ch
        if inc_input:
            self.out_dim += input_ch

    def forward(self, x):
        x = x * np.pi
        sin = [torch.sin(x * freq_band) for freq_band in self.freq_bands]
        cos = [torch.cos(x * freq_band) for freq_band in self.freq_bands]
        oup = torch.cat(sin + cos, -1)
        return oup


class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.

    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a
    # hyperparameter.

    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)

    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first

        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(
                    -1 / self.in_features, 1 / self.in_features
                )
            else:
                self.linear.weight.uniform_(
                    -np.sqrt(6 / self.in_features) / self.omega_0,
                    np.sqrt(6 / self.in_features) / self.omega_0,
                )

    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))

    def forward_with_intermediate(self, input):
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate


class NeRF(nn.Module):
    def __init__(self, args, D=8, W=256, input_ch=4, output_ch=4, skips=[4], multi_res=8):
        super(NeRF, self).__init__()
        self.skips = skips
        self.embedder = Embedder(input_ch, False, multi_res - 1, multi_res)
        input_ch = self.embedder.out_dim

        self.pts_linears = nn.ModuleList(
            [SineLayer(input_ch, W)]
            + [
                SineLayer(W, W)
                if i not in self.skips
                else SineLayer(W + input_ch, W)
                for i in range(D - 1)
            ]
        )

        self.output_linear = nn.Linear(W, output_ch, bias=False)
        with torch.no_grad():
            nn.init.zeros_(self.output_linear.weight)

        moving = sitk.ReadImage(args.moving_vol_path)
        moving = sitk.GetArrayFromImage(moving)
       
        moving = torch.from_numpy(
            (moving - args.min_val) / args.std_val
        ).float()
        self.register_buffer('moving', moving)

    def forward(self, x):
        # print(x.shape)
        x = self.embedder(x)
        identity = x
        for i, linear in enumerate(self.pts_linears):
            x = linear(x)
            if i in self.skips:
                x = torch.cat([identity, x], -1)

        outputs = self.output_linear(x)

        return outputs

    def sample_moving(self, pts):
        for i in range(5 - len(pts.shape)):
            pts = pts.unsqueeze(0)
        vals = F.grid_sample(self.moving[None, None], pts, align_corners=False)
        return vals

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
with open("/content/drive/MyDrive/Colab Notebooks/ETHZ/config.yaml") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
args = SimpleNamespace(**config)

cuda


In [6]:
args

namespace(fixed_vol_path='/content/drive/MyDrive/Colab Notebooks/ETHZ/data/case2_T00-ssm.mha',
          min_val=0,
          std_val=4247,
          num_repeat=1,
          moving_vol_path='/content/drive/MyDrive/Colab Notebooks/ETHZ/data/case2_T50-ssm.mha',
          netdepth=8,
          netwidth=256,
          multires=10,
          num_epochs=4,
          batch_size=2048,
          lr=0.0001,
          lambda_jacob=0.05,
          num_step_opt=1,
          num_step_log=10,
          num_step_val=400,
          num_val_planes=4,
          chunk_size=32768,
          num_epoch_save=1,
          ckpt_nvf_path='/content/drive/MyDrive/Colab Notebooks/ETHZ/check/case2.pth')

In [7]:
dataset = RayDataset(args)
dataloader = DataLoader(
    dataset, batch_size=None, shuffle=True, pin_memory=True
)
dataset.props

[(256, 256, 112),
 (0.0, 0.0, 0.0),
 (1.16, 1.16, 2.5),
 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)]

In [8]:
model = NeRF(args, args.netdepth, args.netwidth, multi_res=args.multires).to(device)
model.load_state_dict(torch.load(args.ckpt_nvf_path)["model"])

<All keys matched successfully>

In [9]:
pts, vals = dataset.all_planes()
H, D, W, _ = pts.shape
pts = pts.reshape(-1, 3).to(device)

prds_i, offs_i = [], []
for j in range(0, dataset.vol_size, args.chunk_size):
    pts_j = pts[j : j + args.chunk_size]
    with torch.no_grad():
        time0 = torch.zeros((len(pts_j), 1)).to(device)

        off_time1 = model(torch.concatenate((pts_j, time0), dim=1))
        tmp1 = pts_j + 1 / 2 * off_time1[:, :3]
        time1 = 1 / 2 * torch.ones((len(pts_j), 1)).to(device)

        off_time2 = model(torch.concatenate((tmp1, time1), dim=1))
        tmp2 = tmp1 + 1 / 2 * off_time2[:, :3]

    new_pts_j = tmp2
    offs_j = new_pts_j - pts_j

    prds_j = model.sample_moving(new_pts_j).view(-1, 1)
    prds_i.append(prds_j)
    offs_i.append(offs_j)

prds_i = torch.cat(prds_i)
offs_i = torch.cat(offs_i)

In [10]:
prds_j.shape, prds_i.shape, offs_i.shape

(torch.Size([32768, 1]), torch.Size([7340032, 1]), torch.Size([7340032, 3]))

In [11]:
prds = prds_i.view(H, D, W)
prds.shape

torch.Size([112, 256, 256])

In [12]:
# prds_img = dataset.tensor2vol(prds.detach(), dvf=False)
# sitk.WriteImage(prds_img, "/content/drive/MyDrive/Colab Notebooks/ETHZ/case2_2.mha")

In [13]:
offs = offs_i.view(H, D, W, 3)
offs_img = dataset.tensor2vol(offs.detach(), dvf=True)
sitk.WriteImage(offs_img, "/content/drive/MyDrive/Colab Notebooks/ETHZ/nvf2.mha")